In [15]:
import z3
from z3 import *

# 1. Mudar para BitVectors de 16 bits (Como pedia o enunciado)
# Isto resolve o problema do "Unknown" porque o espaço de procura é finito.
N = 16

# Declaração das Variáveis
a, b, anext, bnext = BitVecs('a b anext bnext', N)
r, r_ = BitVecs('r r_', N)
s, s_ = BitVecs('s s_', N)
t, t_ = BitVecs('t t_', N)

# Variáveis "Next"
rnext, r_next = BitVecs('rnext r_next', N)
snext, s_next = BitVecs('snext s_next', N)
tnext, t_next = BitVecs('tnext t_next', N)

loop, loopnext = Bools('loop loopnext')
absurd = BoolVal(False)

# Listas
vars = [r, r_, s, s_, t, t_, loop, a, b]
varsnext = [rnext, r_next, snext, s_next, tnext, t_next, loopnext, anext, bnext]

# 2. SFOTS

init = And(
    a > 0, b > 0, 
    r == a, r_ == b, 
    s == 1, s_ == 0, 
    t == 0, t_ == 1, 
    loop
)

# Definimos o Quociente usando SDiv (Signed Division)
# Esta é a forma correta de dividir em BitVectors com sinal
q = r / r_

t0 = And(
    loop,               
    r_ != 0,          

    # Updates das variáveis (Lógica COMPLETA com divisão)
    rnext == r_,
    
    # Resto da divisão (podes usar SRem(r, r_) ou a fórmula manual)
    r_next == r - (q * r_), 
    
    snext == s_,
    s_next == s - (q * s_), # Agora o Z3 consegue resolver isto!
    
    tnext == t_,
    t_next == t - (q * t_), # E isto também!
    
    # Constantes e Flag
    anext == a, bnext == b,
    loopnext == loop     
)

t1 = And(
    loop,
    r_ == 0,
    rnext == r, r_next == r_,
    snext == s, s_next == s_,
    tnext == t, t_next == t_,
    anext == a, bnext == b,
    loopnext == Not(loop)
)

trans = Or(t0, t1)
error = And(loop, r == 0, r_ != 0)

# 3. CHCs
# Truque para criar a assinatura da função sem escrever BitVecSort 10 vezes
bv_sort = BitVecSort(N)
# 8 variáveis BitVec + 1 Bool + (Retorno Bool)
arg_types = [bv_sort] * 8 + [BoolSort()] + [BoolSort()]

Inv = Function('inv', *arg_types)

pre   = Inv(*vars)
pos   = Inv(*varsnext)

chc0 = ForAll(vars, Implies(init, pre))
chc1 = ForAll(vars + varsnext, Implies(And(pre, trans), pos))
chc2 = ForAll(vars, Implies(And(pre, error), absurd))

# 4. Solver 
s = SolverFor('HORN')
s.add([chc0, chc1, chc2])

print("A verificar com BitVectors...")
result = s.check()

if result == sat:
    print("SAT! O programa é SEGURO.")
    # print(s.model().eval(pre))
elif result == unsat:
    print("UNSAT! Erro encontrado.")
else: 
    print("UNKNOWN")
    print(s.reason_unknown())

Z3Exception: Value cannot be converted into a Z3 Boolean value

In [5]:
from z3 import *
import traceback

def check_exa_chc_safe():

    N = 16

    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)
    s, sb    = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]

    absurd = BoolVal(False)

    init = And(
        a > 0, b > 0,
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )

    # Guarded division: define q and constrain apenas quando rb != 0
    q = BitVec('q', N)
    guards = [
        Implies(rb != 0, q == r / rb),
        Implies(rb == 0, q == BitVecVal(0, N))  # valor arbitrário quando rb==0
    ]

    def mul_overflow(x, y):
        ext_x = SignExt(1, x)
        ext_y = SignExt(1, y)
        ext_prod = ext_x * ext_y
        trunc = SignExt(1, x * y)
        return ext_prod != trunc

    def sub_overflow(x, y):
        ext_x = SignExt(1, x)
        ext_y = SignExt(1, y)
        ext_sub = ext_x - ext_y
        trunc = SignExt(1, x - y)
        return ext_sub != trunc

    mult_rnext_overflow = mul_overflow(q, rb)
    mult_snext_overflow = mul_overflow(q, sb)
    mult_tnext_overflow = mul_overflow(q, tb)

    sub_r_overflow = sub_overflow(r, q * rb)
    sub_s_overflow = sub_overflow(s, q * sb)
    sub_t_overflow = sub_overflow(t, q * tb)

    hasOverFlow = Or(
        mult_rnext_overflow,
        mult_snext_overflow,
        mult_tnext_overflow,
        sub_r_overflow,
        sub_s_overflow,
        sub_t_overflow
    )

    t0 = And(on, rb != 0,
        r_p == rb,
        rb_p == r - (q * rb),
        s_p == sb,
        sb_p == s - (q * sb),
        t_p == tb,
        tb_p == t - (q * tb),
        a_p == a, b_p == b,
        on_p == on
    )

    t1 = And(on, rb == 0,
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb,
        a_p == a, b_p == b,
        on_p == Not(on)
    )

    trans = Or(t0, t1)
    error = And(hasOverFlow, on, rb == 0)

    Inv = Function('inv',
        BitVecSort(N), BitVecSort(N),
        BitVecSort(N), BitVecSort(N),
        BitVecSort(N), BitVecSort(N),
        BitVecSort(N), BitVecSort(N),
        BoolSort(),
        BoolSort()
    )

    pre = Inv(*vars_curr)
    pos = Inv(*vars_next)

    chc0 = ForAll(vars_curr, Implies(init, pre))
    chc1 = ForAll(vars_curr + vars_next, Implies(And(pre, trans), pos))
    chc2 = ForAll(vars_curr, Implies(And(pre, error), absurd))

    try:
        s = SolverFor('HORN')
        s.set("timeout", 8000)   # 8 seconds
        s.add(guards)            # adiciona guardas (para evitar divisão descontrolada)
        s.add([chc0, chc1, chc2])
        print("A verificar (safe mode, timeout 8s)...")
        res = s.check()
        print("Resultado:", res)
        if res == sat:
            print(s.model())
    except Exception as e:
        print("Erro ao correr Z3:", type(e), e)
        traceback.print_exc()

check_exa_chc_safe()


A verificar (safe mode, timeout 8s)...
Resultado: unknown


In [15]:
from z3 import *

def check_exa_chc():

    N = 4

    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)  #rb é o r_ do enunciado
    s, sb    = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

  
    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]
    
    absurd = BoolVal(False)

    #SFOTS
    init = And(
        a > 0, b > 0,
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )


    q = UDiv(r, rb) 

    def mul_overflow(x, y):
        return Not(BVMulNoOverflow(x, y, True))

    def sub_overflow(x, y):
        return Or(
            Not(BVSubNoOverflow(x, y)),
            Not(BVSubNoUnderflow(x, y, True))
        )

    mult_rnext_overflow = mul_overflow(q, rb)
    mult_snext_overflow = mul_overflow(q, sb)
    mult_tnext_overflow = mul_overflow(q, tb)

    sub_r_overflow = sub_overflow(r, q * rb)
    sub_s_overflow = sub_overflow(s, q * sb)
    sub_t_overflow = sub_overflow(t, q * tb)

    hasOverFlow = Or(
        mult_rnext_overflow,
        mult_snext_overflow,
        mult_tnext_overflow,
        sub_r_overflow,
        sub_s_overflow,
        sub_t_overflow
    )




    # t0 é o começo do loop
    t0 = And(on, rb != 0,         
        r_p == rb,
        rb_p == r - (q * rb),
        s_p == sb,              # Todo o calculo 
        sb_p == s - (q * sb),
        t_p == tb,
        tb_p == t - (q * tb),

        a_p == a, b_p == b,
        on_p == on        # As variaveis que nao mudam 
    )

    # saida do loop
    t1 = And( on,rb == 0,          
        
        
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb, # As variaveis que nao mudam 
        a_p == a, b_p == b,
        
        on_p == Not(on)   # o loop acabou
    )

    trans = Or(t0, t1)

    # ERROR
    error = Or(r == 0,hasOverFlow)

    

    # funçao inv que tem as variaveis ints e o bool do loop e retorna um bool
    Inv = Function('inv',  BitVecSort(N), BitVecSort(N), BitVecSort(N),
    BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N), BitVecSort(N),BoolSort(), 
    BoolSort())

    pre = Inv(*vars_curr)
    pos = Inv(*vars_next)
    

    # CHC 1: Init => Invariante
    chc0 = ForAll(vars_curr, Implies(init, pre))

    # CHC 2: (Invariante AND Transição) => Invariante'
    chc1 = ForAll(vars_curr + vars_next, Implies(And(pre, trans), pos))

    # CHC 3: (Invariante AND Erro) => False (Absurdo)
    chc2 = ForAll(vars_curr, Implies(And(pre, error), absurd))


    s = SolverFor('HORN')
    s.add([chc0, chc1, chc2])
    
    print("A verificar segurança (r != 0)...")
    result = s.check()
    
    if result == sat:
        print("SAFE! O programa é seguro (r nunca é 0 indevidamente).")
        print("Esqueça Isto não é pra si!!")
        print(s.model().eval(pre)) 
    elif result == unsat:
        print("UNSAFE! Encontrado contra-exemplo.")
        #print(s.proof())
    else:
        print("UNKNOWN.")
        print(s.reason_unknown())

check_exa_chc()

A verificar segurança (r != 0)...
UNKNOWN.
interrupted from keyboard


In [14]:
from z3 import *

def check_exa_chc_optimized():
    # N=4 é muito pequeno para Euclides Estendido sem overflow, 
    # mas serve para verificar a lógica do resto (GCD).
    N = 4 

    # Dica: Usar BitVecSort explicitamente ajuda na leitura
    BVS = BitVecSort(N)
    
    a, b     = BitVecs('a b', N)
    r, rb    = BitVecs('r rb', N)
    s, sb    = BitVecs('s sb', N)
    t, tb    = BitVecs('t tb', N)
    on       = Bool('on')

    a_p, b_p   = BitVecs('a_p b_p', N)
    r_p, rb_p  = BitVecs('r_p rb_p', N)
    s_p, sb_p  = BitVecs('s_p sb_p', N)
    t_p, tb_p  = BitVecs('t_p tb_p', N)
    on_p       = Bool('on_p')

    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]
    
    absurd = BoolVal(False)

    # Init: Garante que a e b são positivos para evitar GCD(0,0)
    init = And(
        UGT(a, 0), UGT(b, 0), # Usar UGT (Unsigned Greater Than) é mais seguro
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )

    # OTIMIZAÇÃO: Calcular q e resto
    # UDiv é Unsigned Division. 
    # Em Python Z3, '/' em BitVec é UDiv, mas ser explicito ajuda.
    q = UDiv(r, rb) 
    
    # Transição do Loop (t0)
    t0 = And(
        on, 
        rb != 0,         
        r_p == rb,
        # OTIMIZAÇÃO: Usar URem (Unsigned Remainder) é mais eficiente que (r - q*rb)
        rb_p == URem(r, rb), 
        
        s_p == sb,
        sb_p == s - (q * sb), # Aqui mantemos a aritmética original para s e t
        t_p == tb,
        tb_p == t - (q * tb),

        a_p == a, b_p == b,
        on_p == on
    )

    # Saída do Loop (t1)
    t1 = And(
        on, 
        rb == 0,
        
        # O estado mantém-se igual, só muda o boleano de controlo
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb,
        a_p == a, b_p == b,
        
        on_p == Not(on) # Sai do loop
    )

    trans = Or(t0, t1)

    # ERROR: Vamos focar apenas na correção do algoritmo (GCD != 0)
    # Se quiseres verificar Overflow, terás de aumentar N, 
    # pois com N=4 vai dar UNSAFE quase sempre.
    error = And(Not(on), r == 0) 

    # Declaração da Função Invariante
    Inv = Function('inv', *[s.sort() for s in vars_curr] + [BoolSort()])

    pre = Inv(*vars_curr)
    pos = Inv(*vars_next)

    # CHC System
    chc0 = ForAll(vars_curr, Implies(init, pre))
    chc1 = ForAll(vars_curr + vars_next, Implies(And(pre, trans), pos))
    chc2 = ForAll(vars_curr, Implies(And(pre, error), absurd))

    s = SolverFor('HORN')
    s.add([chc0, chc1, chc2])
    
    print(f"A verificar lógica GCD para N={N}...")
    result = s.check()
    
    if result == sat:
        print("SAFE! O algoritmo termina sempre com r != 0.")
        print(s.model().eval(pre)) 
    elif result == unsat:
        print("UNSAFE! Encontrado contra-exemplo (r pode ser 0).")
    else:
        print("UNKNOWN.")
        print(s.reason_unknown())

check_exa_chc_optimized()

A verificar lógica GCD para N=4...
SAFE! O algoritmo termina sempre com r != 0.
Not(r == 0)


In [16]:
from z3 import *

def check_exa_chc_overflow_fast():
    # --- CONFIGURAÇÃO ---
    N = 4  # Número de bits
    
    # Limites para verificar Overflow como se fossem BitVectors
    # a, b, r são Unsigned (0 até 2^N - 1)
    MAX_UINT = (2 ** N) - 1 
    
    # s, t são Signed (Complemento para 2: -2^(N-1) até 2^(N-1) - 1)
    MAX_INT = (2 ** (N - 1)) - 1
    MIN_INT = -(2 ** (N - 1))
    
    print(f"Verificando N={N} bits.")
    print(f"Limites Signed (s, t): [{MIN_INT}, {MAX_INT}]")
    print(f"Limites Unsigned (r): [0, {MAX_UINT}]")
    print("-" * 30)

    # --- VARIÁVEIS (Usando IntSort para velocidade) ---
    a, b     = Ints('a b')
    r, rb    = Ints('r rb')
    s, sb    = Ints('s sb')
    t, tb    = Ints('t tb')
    on       = Bool('on')

    a_p, b_p   = Ints('a_p b_p')
    r_p, rb_p  = Ints('r_p rb_p')
    s_p, sb_p  = Ints('s_p sb_p')
    t_p, tb_p  = Ints('t_p tb_p')
    on_p       = Bool('on_p')

    vars_curr = [a, b, r, rb, s, sb, t, tb, on]
    vars_next = [a_p, b_p, r_p, rb_p, s_p, sb_p, t_p, tb_p, on_p]
    
    # --- RESTRIÇÕES DE TIPO (Simular BitVector) ---
    # Obrigamos o Z3 a considerar apenas inputs válidos para N bits
    valid_range = And(
        a > 0, a <= MAX_UINT,
        b > 0, b <= MAX_UINT
    )

    # --- INICIALIZAÇÃO ---
    init = And(
        valid_range,
        r == a, rb == b,
        s == 1, sb == 0,
        t == 0, tb == 1,
        on
    )

    # --- CÁLCULO DA TRANSIÇÃO ---
    # O Z3 resolve divisões em Ints instantaneamente
    q = r / rb  # Divisão inteira
    
    # Valores previstos para o próximo estado (matemáticos)
    val_sb_next = s - (q * sb)
    val_tb_next = t - (q * tb)
    
    # Condição de Overflow: Se o próximo valor sair dos limites de N bits
    s_overflow = Or(val_sb_next > MAX_INT, val_sb_next < MIN_INT)
    t_overflow = Or(val_tb_next > MAX_INT, val_tb_next < MIN_INT)
    
    # Overflow na multiplicação intermédia (q * sb) também conta? 
    # Normalmente em assembly sim, mas vamos focar no resultado armazenado.
    hasOverFlow = Or(s_overflow, t_overflow)

    # Transição do Loop (t0)
    t0 = And(
        on, 
        rb != 0,
        
        # Lógica do algoritmo
        r_p == rb,
        rb_p == r % rb,  # Modulo em Int
        
        s_p == sb,
        sb_p == val_sb_next, 
        
        t_p == tb,
        tb_p == val_tb_next,

        a_p == a, b_p == b,
        on_p == on
    )

    # Saída do Loop (t1)
    t1 = And(
        on, 
        rb == 0,
        
        r_p == r, rb_p == rb,
        s_p == s, sb_p == sb,
        t_p == t, tb_p == tb,
        a_p == a, b_p == b,
        
        on_p == Not(on)
    )

    trans = Or(t0, t1)

    # --- PROPRIEDADE DE ERRO ---
    # O sistema falha se detectarmos Overflow OU se o GCD for 0 (impossível se a,b>0)
    # Nota: Colocamos o overflow aqui para ele detectar se ALGUM passo viola os limites
    error = Or(r == 0, hasOverFlow)

    # --- HORN CLAUSES ---
    Inv = Function('inv', *[IntSort() for _ in vars_curr] + [BoolSort(), BoolSort()])
    pre = Inv(*vars_curr)
    pos = Inv(*vars_next)

    # CHC 1: Init => Inv
    chc0 = ForAll(vars_curr, Implies(init, pre))

    # CHC 2: Inv /\ Trans => Inv'
    # Importante: Adicionamos !hasOverflow na pré-condição da transição
    # para garantir que só continuamos se não houve erro antes.
    chc1 = ForAll(vars_curr + vars_next, 
                  Implies(And(pre, trans, Not(hasOverFlow)), pos))

    # CHC 3: Inv /\ Error => False
    chc2 = ForAll(vars_curr, Implies(And(pre, error), BoolVal(False)))

    s = SolverFor('HORN')
    s.add([chc0, chc1, chc2])
    
    print("A verificar segurança com deteção de Overflow...")
    result = s.check()
    
    if result == sat:
        print("SAFE! Não existem inputs de 4 bits que causem overflow.")
    elif result == unsat:
        print("UNSAFE! O algoritmo DEU OVERFLOW.")
        print("Isto é esperado para N=4, pois os coeficientes de Bézout crescem rápido.")
        
        # Vamos tentar extrair um contra-exemplo simples se possível
        # (Proof extraction em CHC é complexo, mas o status já confirma)
    else:
        print("UNKNOWN.")
        print(s.reason_unknown())

check_exa_chc_overflow_fast()

Verificando N=4 bits.
Limites Signed (s, t): [-8, 7]
Limites Unsigned (r): [0, 15]
------------------------------


Z3Exception: b'Wrong number of arguments (9) passed to function (declare-fun inv (Int Int Int Int Int Int Int Int Int Bool) Bool) \narg: a\n\narg: b\n\narg: r\n\narg: rb\n\narg: s\n\narg: sb\n\narg: t\n\narg: tb\n\narg: (ite on 1 0)\n'